In [1]:
# OHLCV + VQ-VAE 코드 + 뉴스 기반 시황 LLM 파인튜닝 (Llama 3.1 8B, SageMaker JumpStart)

이 노트북에서는 다음 파이프라인을 구현합니다.

1. OHLCV + VQ-VAE 코드 + 뉴스 + 시황 텍스트 라벨이 담긴 CSV를 로드
2. SageMaker JumpStart의 instruction-based fine-tuning 포맷(JSONL + template.json)으로 변환
3. S3에 업로드
4. Meta Llama 3.1 8B Instruct 모델을 LoRA 방식으로 파인튜닝
5. 엔드포인트로 배포 후, "코드 + 요약 + 뉴스"를 입력하면 한국어 시황 설명을 생성


SyntaxError: invalid decimal literal (3061262297.py, line 8)

In [2]:
# 1. 필수 라이브러리 설치 (Studio 기본 이미지면 대부분 이미 설치되어 있음)
# 필요시만 주석 해제해서 실행
# !pip install "sagemaker>=2.218.0" "boto3" "pandas" "pyarrow"

import os
import json
import boto3
import pandas as pd

import sagemaker
from sagemaker.s3 import S3Uploader
from sagemaker.jumpstart.estimator import JumpStartEstimator  # JumpStart 전용 Estimator

sess = sagemaker.Session()
region = sess.boto_region_name
role = sagemaker.get_execution_role()

print("Region:", region)
print("Role:", role)
print("Default bucket:", sess.default_bucket())


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Region: us-west-2
Role: arn:aws:iam::327784329358:role/SageMakerExecutionRole-KoreaUniv-DA
Default bucket: sagemaker-us-west-2-327784329358


In [3]:
# 2. 데이터 로드
# 예시: data/market_commentary_train.csv
# 컬럼 예시:
#   - code_id: VQ-VAE 코드 (예: "z17")
#   - ohlcv_summary: "시가 492.1, 고가 500.2, 저가 480.3, 종가 495.7, 거래량 30일 평균 대비 1.8배"
#   - news_summary: "FOMC 이후 금리 동결, 향후 인하 가능성 시사"
#   - commentary_ko: 사람이 쓴 한국어 시황 설명

train_csv_path = "data/market_commentary_train.csv"
val_csv_path = "data/market_commentary_val.csv"  # 없다면 None 처리 가능

train_df = pd.read_csv(train_csv_path)
print("Train shape:", train_df.shape)
train_df.head()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 train_csv_path = "data/market_commentary_train.csv"                                         │
│   10 val_csv_path = "data/market_commentary_val.csv"  # 없다면 None 처리 가능                    │
│   11                                                                                             │
│ ❱ 12 train_df = pd.read_csv(train_csv_path)                                                      │
│   13 print("Train shape:", train_df.shape)                                                       │
│   14 train_df.head()                                                                             │
│   15                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pandas/io/parsers/reader │
│ s.py:1026 in read_csv                                                                            │
│                                                                                                  │
│   1023 │   )                                                                                     │
│   1024 │   kwds.update(kwds_defaults)                                                            │
│   1025 │                                                                                         │
│ ❱ 1026 │   return _read(filepath_or_buffer, kwds)                                                │
│   1027                                                                                           │
│   1028                                                                                           │
│   1029 # iterator=True -> TextFileReader                                                         │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pandas/io/parsers/reader │
│ s.py:620 in _read                                                                                │
│                                                                                                  │
│    617 │   _validate_names(kwds.get("names", None))                                              │
│    618 │                                                                                         │
│    619 │   # Create the parser.                                                                  │
│ ❱  620 │   parser = TextFileReader(filepath_or_buffer, **kwds)                                   │
│    621 │                                                                                         │
│    622 │   if chunksize or iterator:                                                             │
│    623 │   │   return parser                                                                     │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pandas/io/parsers/reader │
│ s.py:1620 in __init__                                                                            │
│                                                                                                  │
│   1617 │   │   │   self.options["has_index_names"] = kwds["has_index_names"]                     │
│   1618 │   │                                                                                     │
│   1619 │   │   self.handles: IOHandles | None = None                                             │
│ ❱ 1620 │   │   self._engine = self._make_engine(f, self.engine)   

In [4]:
# 3. CSV -> JSONL 변환 유틸리티

def df_to_jsonl(df: pd.DataFrame, jsonl_path: str):
    """
    DataFrame에서 JumpStart instruction SFT용 jsonl 파일 생성.
    template.json에서 요구하는 필드는 {code, ohlcv_summary, news_summary, commentary}.
    """
    required_cols = ["code_id", "ohlcv_summary", "news_summary", "commentary_ko"]
    for c in required_cols:
        if c not in df.columns:
            raise ValueError(f"필수 컬럼 {c} 이(가) DataFrame에 없습니다.")

    with open(jsonl_path, "w", encoding="utf-8") as f:
        for row in df.itertuples():
            record = {
                "code": getattr(row, "code_id"),
                "ohlcv_summary": getattr(row, "ohlcv_summary"),
                "news_summary": getattr(row, "news_summary"),
                "commentary": getattr(row, "commentary_ko"),
            }
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

    print(f"Saved JSONL: {jsonl_path}")


os.makedirs("prepared_data", exist_ok=True)
train_jsonl = "prepared_data/train.jsonl"
df_to_jsonl(train_df, train_jsonl)

# 검증용 데이터가 있다면 동일하게 처리
val_jsonl = None
try:
    val_df = pd.read_csv(val_csv_path)
    val_jsonl = "prepared_data/val.jsonl"
    df_to_jsonl(val_df, val_jsonl)
except FileNotFoundError:
    print("검증용 CSV를 찾지 못했습니다. validation은 training 데이터에서 자동 split하게 설정 예정.")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:28                                                                                   │
│                                                                                                  │
│   25                                                                                             │
│   26 os.makedirs("prepared_data", exist_ok=True)                                                 │
│   27 train_jsonl = "prepared_data/train.jsonl"                                                   │
│ ❱ 28 df_to_jsonl(train_df, train_jsonl)                                                          │
│   29                                                                                             │
│   30 # 검증용 데이터가 있다면 동일하게 처리                                                      │
│   31 val_jsonl = None                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_df' is not defined

In [5]:
template = {
    "prompt": (
        "당신은 한국 주식 및 지수 시장을 분석하는 애널리스트입니다.\n"
        "아래의 VQ-VAE 코드, OHLCV 요약, 뉴스 헤드라인을 기반으로 "
        "해당 날짜의 시황을 한국어로 자세히 설명하세요.\n\n"
        "### 코드\n{code}\n\n"
        "### OHLCV 요약\n{ohlcv_summary}\n\n"
        "### 뉴스 요약\n{news_summary}\n"
    ),
    "completion": "{commentary}",
}

template_path = "prepared_data/template.json"
with open(template_path, "w", encoding="utf-8") as f:
    json.dump(template, f, ensure_ascii=False, indent=2)

print("Saved template:", template_path)


Saved template: prepared_data/template.json


In [6]:
# 5. JumpStartEstimator 설정

model_id = "meta-textgeneration-llama-3-1-8b-instruct"  # AWS 블로그 표 참고
training_instance_type = "ml.g5.12xlarge"  # 블로그에서 권장 인스턴스 타입 예시

# EULA 동의 (필수)
environment = {
    "accept_eula": "true"  # 처음 fine-tune 시 EULA 수락 필요
}

estimator = JumpStartEstimator(
    model_id=model_id,
    role=role,
    instance_type=training_instance_type,
    environment=environment,
    # output_path 를 지정하고 싶으면 아래처럼 설정 가능
    # output_path=f"s3://{bucket}/{prefix}/output",
)

# 하이퍼파라미터 설정 (기본값에서 약간만 바꾼 예시)
# 실제로는 블로그/예제 notebook의 hyperparameter 표를 참고해서 조정하는게 좋음
hyperparameters = {
    # LoRA/QLoRA 관련은 JumpStart 기본 설정을 따르고,
    # 여기서는 epoch, max_input_length, batch_size 정도만 건드린다고 가정
    "epoch": "2",
    "max_input_length": "2048",
    "per_device_train_batch_size": "4",
    # validation이 따로 없으면 train에서 일부를 자동 split
    "validation_split_ratio": "0.1" if val_s3_uri is None else "0.0",
}

estimator.set_hyperparameters(**hyperparameters)
print("Estimator configured.")


Using model 'meta-textgeneration-llama-3-1-8b-instruct' with wildcard version identifier '*'. You can pin to version '2.17.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:29                                                                                   │
│                                                                                                  │
│   26 │   "max_input_length": "2048",                                                             │
│   27 │   "per_device_train_batch_size": "4",                                                     │
│   28 │   # validation이 따로 없으면 train에서 일부를 자동 split                                  │
│ ❱ 29 │   "validation_split_ratio": "0.1" if val_s3_uri is None else "0.0",                       │
│   30 }                                                                                           │
│   31                                                                                             │
│   32 estimator.set_hyperparameters(**hyperparameters)                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'val_s3_uri' is not defined

In [ ]:
# 6. 학습 실행
# - "training" 채널에 우리가 올린 train 폴더 S3 URI를 전달
# - validation 채널은 JumpStart SFT 스펙에 따라 필요시 추가 (모델/버전에 따라 다름)

input_channels = {"training": train_s3_uri}
# 일부 모델은 {"validation": val_s3_uri} 도 지원하므로, 사용하는 model_id의 예제를 확인해서 맞춰야 함

estimator.fit(input_channels)
